In [1]:
import numpy as np
import pandas as pd
from numpy import *
from scipy.optimize import *

In [2]:
### internal functions:

# 0. Calcilation of dewpoint temperature, T_dp,C
def T_dp(T, RH):
    T_dp = 257.14*(np.log(RH/100*np.exp((18.678-T/234.5)*(T/(257.14+T)))))/(18.678-(np.log(RH/100*np.exp((18.678-T/234.5)*(T/(257.14+T))))))
    return T_dp

# 1. Calculation of Wetbulb temperature: T_wb, C
# a. first step:
def E_wb(T_wb,P_atm):
    #Satuation pressure of water at wetbulb temperature
    P_sat_twb = 611.2 * np.exp(((18.678 - T_wb/234.5) * T_wb)/(T_wb + 257.14))
    #Absolute humidity at saturation and t_wb
    d_sat_twb = 0.62198 * P_sat_twb/(P_atm - P_sat_twb)
    #Enthalpy at temperature of T_wb
    E_wb = 1.01 * T_wb + (2500 + 1.84 * T_wb) * d_sat_twb
    return E_wb
# b.second step:
def wet_blub(T_oa,RH_oa,P_atm,P_sat):
    #vapor pressure
    P_vapor = P_sat * RH_oa/100
    #absolute humidity
    d_oa = 0.62198 * P_vapor/(P_atm - P_vapor)
    #Enthalpy at temperature of T_oa
    E_oa = 1.01 * T_oa + (2500 + 1.84 * T_oa) * d_oa
    y = lambda t: E_wb(t,P_atm) - E_oa
    initial_guess = -20
    T_wb = fsolve(y,initial_guess)[0]
    return T_wb

# 2. In airside economizer, the calculation of Supply air drybulb temperature: T_sa, C
def T_supplyair(T_sa_lower,T_sa_upper,T_oa):
    if T_oa < T_sa_lower:
        T_sa = T_sa_lower
    elif T_oa > T_sa_upper:
        T_sa = T_sa_upper
    else:
        T_sa = T_oa
    return T_sa

# 3. The calculation of lower and upper bound of absolute humidity: g/kg
def Supply_humidity_boundary(T_sa,P_atm,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper):   
    P_sat = 611.21 * np.exp(((18.678 - T_sa/234.5) * T_sa)/(T_sa + 257.14))
    # lower boundary
    P_act_l = 611.21 * np.exp((dp_sa_lower * 18.678)/(257.14 + dp_sa_lower))
    RH_l = P_act_l/P_sat
    if RH_l >= RH_sa_lower and RH_l <= RH_sa_upper:
        d_lower = 621.98 * P_act_l/(P_atm - P_act_l)
    elif RH_l > RH_sa_upper:
        d_lower = 621.98 * (P_sat * RH_sa_upper)/(P_atm - P_sat * RH_sa_upper)
    else:
        d_lower = 621.98 * (P_sat * RH_sa_lower)/(P_atm - P_sat * RH_sa_lower)
    # upper boundary
    P_act_u = 611.21 * np.exp((dp_sa_upper * 18.678)/(257.14 + dp_sa_upper))
    RH_u = P_act_u/P_sat
    if RH_u >= RH_sa_lower and RH_u <= RH_sa_upper:
        d_upper = 621.98 * P_act_u/(P_atm - P_act_u)
    elif RH_u > RH_sa_upper:
        d_upper = 621.98 * (P_sat * RH_sa_upper)/(P_atm - P_sat * RH_sa_upper)
    else:
        d_upper = 621.98 * (P_sat * RH_sa_lower)/(P_atm - P_sat * RH_sa_lower)
    return d_lower,d_upper

# 4. Airside economizer:
def Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper): 
    d_threshold_1 = Supply_humidity_boundary(27,P_atm,-9,15,0,15)[1]
    E_threshold_1 = 1.01 * 27 + (2500 + 1.84 * 27) * d_threshold_1/1000
    if E_oa < E_threshold_1:
        AE_use = 1
        E_oa_cd = E_oa
        E_sa = E_oa
        if T_oa <= T_sa:
            P_sat_threshold_11 = 611.21 * np.exp(((18.678 - T_sa/234.5) * T_sa)/(T_sa + 257.14))
            d_threshold_11 = 621.98 * (P_sat_threshold_11 * 20/100)/(P_atm - P_sat_threshold_11 * 20/100)
            if d_oa >= d_threshold_11:
                HD_use = 0
                hd_g_per_kg = 0
                T_oa_cd = T_oa
                d_oa_cd = d_oa
                d_sa = d_oa_cd
            else:
                HD_use = 1
                d_oa_cd = d_threshold_11
                d_sa = d_threshold_11
                hd_g_per_kg = max(d_threshold_11 - d_oa,0)
                T_oa_cd = (E_sa-2.5*d_oa_cd)/(1.01+1.84*d_oa_cd/1000)
        else:
            HD_use = 1
            T_oa_cd = T_sa
            d_oa_cd = (E_oa_cd - 1.01*T_oa_cd)*1000/(2500+1.84*T_oa_cd)
            d_sa = d_oa_cd
            hd_g_per_kg = max(d_oa_cd - d_oa,0)
    else:
        P_sat_threshold_2 = 611.21 * np.exp(((18.678 - T_sa/234.5) * T_sa)/(T_sa + 257.14))
        d_threshold_2 = 621.98 * (P_sat_threshold_2 * 80/100)/(P_atm - P_sat_threshold_2 * 80/100)
        #d_threshold_2 = 621.98 * (P_sat_threshold_2 * 90/100)/(P_atm - P_sat_threshold_2 * 90/100)
        E_threshold_2 = 1.01 * T_sa + (2500 + 1.84 * T_sa) * d_threshold_2/1000
        if E_oa <= E_threshold_2:
            E_oa_cd = E_oa
            AE_use = 1
            HD_use = 1
            E_sa = E_oa
            T_oa_cd = T_sa
            d_oa_cd = (E_oa_cd - 1.01*T_oa_cd)*1000/(2500+1.84*T_oa_cd)
            d_sa = d_oa_cd
            hd_g_per_kg = max(d_oa_cd - d_oa,0)
        else:
            AE_use = 0
            HD_use = 0
            hd_g_per_kg = 0
            d_sa_lower = Supply_humidity_boundary(T_sa,P_atm,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[0]
            d_sa_upper = Supply_humidity_boundary(T_sa,P_atm,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[1]
            d_sa = (d_sa_lower+d_sa_upper)/2
            E_sa = 1.01 * T_sa + (2500 + 1.84 * T_sa) * d_sa/1000
            T_oa_cd = T_sa #fake number, this won't change results when multiply by AE_use
            E_oa_cd = E_sa #fake number, this won't change results when multiply by AE_use
    return AE_use,HD_use,hd_g_per_kg,T_oa_cd,E_oa_cd,E_sa,d_sa

# 5. Air density, kg/m^3
def air_density(P_atm,T_air):
    density = 1.293*(P_atm/101325)*(273.15/(273.15+T_air))
    return density

# 6. Fan power, kW
def Fan_Power(mass_flow_rate,air_density,Fan_Pressure,Fan_e):
    power = mass_flow_rate/air_density*Fan_Pressure/Fan_e/1000
    return power

# 7. Pump power, kW
def Pump_Power(mass_flow_rate,Pump_Pressure,Pump_e,liquid_density):
    power = Pump_Pressure*mass_flow_rate/(1000*Pump_e*liquid_density)
    return power


# 8. Latent heat of vaporization of water: H_water, kj/kg
def Latent_heat_vaporization(T_water):
    H_water = -0.0013*(T_water)**(2)-2.3097*(T_water)+2500.5
    return H_water

# 9. In waterside economizer, the calculation of supply facility water temperature: T_sfw, (chiller system can also use this one if want)
def T_supplywater(T_sa,T_ra,HTE):
    T_sfw = T_ra - (T_ra - T_sa)/HTE
    return T_sfw

# 10. Usage scenario of waterside economizer: 1--use, 0--not use
def Waterside_economizer_use(T_rfw,T_wb_oa,AT_CT,AT_HE):
    if T_wb_oa + AT_CT + AT_HE <= T_rfw:
        use = 1
    else:
        use = 0
    return use

# 11. The heat removed by waterside economizer: WE_heat_removed, kW
def Water_heat_removed(Cooling_required,T_sfw,T_rfw,T_wb_oa,AT_CT,AT_HE):
    if (T_wb_oa + AT_CT + AT_HE) <= T_sfw:
        WE_heat_removed = Cooling_required
    elif (T_wb_oa + AT_CT + AT_HE) > T_sfw and (T_wb_oa + AT_CT + AT_HE) < T_rfw:
        WE_heat_removed = Cooling_required*((T_rfw-(T_wb_oa + AT_CT + AT_HE))/(T_rfw-T_sfw))
    else:
        WE_heat_removed = 0
    return WE_heat_removed

# 12. Air or liquid cooling requirements
def Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage):
    # Percentage of air cooling:
    AC_percentage = 100/100
    # Percentage of liquid cooling:
    LC_percentage = 1 - AC_percentage
    
    Powerloss_UPS = Power_IT/UPS_e - Power_IT
    Powerloss_PD = Power_IT/(1-PD_lr) - Power_IT
    Power_Lighting = Power_IT * L_percentage
    Heat_people = 0
    # Total heat generaton of data center:
    DC_heat = Power_IT + Powerloss_UPS + Powerloss_PD + Power_Lighting + Heat_people
    # Cooling supplied by air: DC_heat_AC, kW (use air to cool IT devices)
    DC_heat_AC = DC_heat * AC_percentage
    # Cooling supplied by air: DC_heat_LC, kW (use liquid to cool IT devices)
    DC_heat_LC = DC_heat * LC_percentage
    return Powerloss_UPS,Powerloss_PD,Power_Lighting,Heat_people,DC_heat_AC,DC_heat_LC,DC_heat

# 13. COP of Chiller, kW/kW (regression)
def COP_Chiller(load, t):
    COP = 16.798751 +  0.008840*t**2 - 8.528931*load**2 + 9.885294*load - 0.760621*t + 0.084615*load*t 
    return COP